# AFL Model - Part 4 - Weekly Predictions
Now that we have explored different algorithms for modelling, we can implement our chosen model and predict this week's AFL games! All you need to do is run the afl_modelling script each Thursday or Friday to predict the following week's games.

In [25]:
# Import Modules
from afl_feature_creation import prepare_afl_features
import afl_data_cleaning
import afl_feature_creation
import afl_modelling
import datetime
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

## Creating The Features For This Weekend's Games
To actually predict this weekend's games, we need to create the same features that we have created in the previous tutorials for the games that will be played this weekend. This includes all the rolling averages, efficiency features, elo features etc. So the majority of this tutorial will be using previously defined functions to create features for the following weekend's games.

### Create Next Week's DataFrame
Let's first get our cleaned afl_data dataset, as well as the odds for next weekend and the 2018 fixture.

In [26]:
# Grab the cleaned AFL dataset and the column order
afl_data = afl_data_cleaning.prepare_afl_data()
ordered_cols = afl_data.columns

# Define a function which grabs the odds for each game for the following weekend
def get_next_week_odds(path):
    # Get next week's odds
    next_week_odds = pd.read_csv(path)
    next_week_odds = next_week_odds.rename(columns={"team_1": "home_team", 
                                                "team_2": "away_team", 
                                                "team_1_odds": "odds", 
                                                "team_2_odds": "odds_away"
                                               })
    return next_week_odds

# Import the fixture
# Define a function which gets the fixture and cleans it up
def get_fixture(path):
    # Get the afl fixture
    fixture = pd.read_csv(path)

    # Replace team names and reformat
    fixture = fixture.replace({'Brisbane Lions': 'Brisbane', 'Footscray': 'Western Bulldogs'})
    fixture['Date'] = pd.to_datetime(fixture['Date']).dt.date.astype(str)
    fixture = fixture.rename(columns={"Home.Team": "home_team", "Away.Team": "away_team"})
    return fixture

next_week_odds = get_next_week_odds("data/weekly_odds.csv")
fixture = get_fixture("data/afl_fixture_2018.csv")

In [27]:
fixture.tail()

,Date,Season,Season.Game,Round,home_team,away_team,Venue
211,2018-08-24,2018,212,NaN,Brisbane,West Coast,Gabba
212,2018-08-24,2018,213,NaN,Carlton,St Kilda,Etihad Stadium
213,2018-08-24,2018,214,NaN,Richmond,Melbourne,MCG
214,2018-08-24,2018,215,NaN,Port Adelaide,Essendon,Adelaide Oval
215,2018-08-24,2018,216,NaN,Fremantle,Collingwood,Optus Stadium


Now that we have these DataFrames, we will define a function which combines the fixture and next week's odds to create a single DataFrame for the games over the next 7 days. To use this function we will need Game IDs for next week. So we will create another function which creates Game IDs by using the Game ID from the last game played and adding 1 to it.

In [28]:
# Define a function which creates game IDs for this week's footy games
def create_next_weeks_game_ids(afl_data):
    odds = get_next_week_odds("data/weekly_odds.csv")

    # Get last week's Game ID
    last_afl_data_game = afl_data['Game'].iloc[-1]

    # Create Game IDs for next week
    game_ids = [(i+1) + last_afl_data_game for i in range(odds.shape[0])]
    return game_ids

# Define a function which creates this week's footy game DataFrame
def get_next_week_df(afl_data):
    # Get the fixture and the odds for next week's footy games
    fixture = get_fixture("data/afl_fixture_2018.csv")
    next_week_odds = get_next_week_odds("data/weekly_odds.csv")
    next_week_odds['Game'] = create_next_weeks_game_ids(afl_data)

    # Get today's date and next week's date and create a DataFrame for next week's games
    todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
    date_in_7_days = (datetime.datetime.today() + datetime.timedelta(days=7)).strftime('%Y-%m-%d')
    fixture = fixture[(fixture['Date'] >= todays_date) & (fixture['Date'] < date_in_7_days)].drop(columns=['Season', 'Season.Game', 'Round'])
    next_week_df = pd.merge(fixture, next_week_odds, on=['home_team', 'away_team'])
    # Split the DataFrame onto two rows for each game
    h_df = next_week_df[['Date', 'Game', 'home_team', 'away_team', 'odds']]
    h_df['Team'] = h_df['home_team']
    h_df['Opposition'] = h_df['away_team']
    h_df['Home?'] = 1
    a_df = next_week_df[['Date', 'Game', 'home_team', 'away_team', 'odds_away']].rename(columns={'odds_away': 'odds'})
    a_df['Team'] = a_df['away_team']
    a_df['Opposition'] = a_df['home_team']
    a_df['Home?'] = 0
    next_week = a_df.append(h_df).sort_values(by='Game')
    return next_week

In [29]:
next_week_df = get_next_week_df(afl_data)
game_ids_next_round = create_next_weeks_game_ids(afl_data)
next_week_df.head()

,Date,Game,home_team,away_team,odds,Team,Opposition,Home?
0,2018-08-03,15363,Richmond,Geelong,3.10,Geelong,Richmond,0
0,2018-08-03,15363,Richmond,Geelong,1.46,Richmond,Geelong,1
1,2018-08-04,15364,Hawthorn,Essendon,1.88,Essendon,Hawthorn,0
1,2018-08-04,15364,Hawthorn,Essendon,2.14,Hawthorn,Essendon,1
2,2018-08-04,15365,Brisbane,North Melbourne,2.18,North Melbourne,Brisbane,0


### Create Each Feature
Now let's append next week's DataFrame to our afl_data DataFrame and then create all the features we used in the [AFL Feature Creation Tutorial](0.2. afl_feature_creation_tutorial.ipynb).

In [30]:
# Append next week's games to our afl_data DataFrame
afl_data = afl_data.append(next_week_df).reset_index(drop=True)
afl_data = afl_data[ordered_cols]

# Create disposal efficiency column
afl_data['disposal_efficiency'] = afl_data['ED'] / afl_data['D']

# Create rolling averages for our statistics
cols_indx_start = afl_data.columns.get_loc("GA")
afl_avgs = afl_feature_creation.create_rolling_averages(afl_data, 6, afl_data.columns[cols_indx_start:])

# Create form between teams feature
afl_avgs = afl_feature_creation.form_between_teams(afl_avgs, 6)

# Apply elos
elos, probs, elo_dict = afl_feature_creation.elo_applier(afl_data, 24)
afl_avgs['home_elo'] = afl_avgs['Game'].map(elos).str[0]
afl_avgs['away_elo'] = afl_avgs['Game'].map(elos).str[1]

# Get elos for next week
afl_avgs.loc[afl_avgs['home_elo'].isna(), 'home_elo'] = afl_avgs[afl_avgs['home_elo'].isna()]['home_team'].map(elo_dict)
afl_avgs.loc[afl_avgs['away_elo'].isna(), 'away_elo'] = afl_avgs[afl_avgs['away_elo'].isna()]['away_team'].map(elo_dict)

# Create Adjusted Margin and then Average it over a 6 game window
afl_avgs = afl_feature_creation.map_elos(afl_avgs)
afl_avgs['Adj_elo_ave_margin'] = afl_avgs['Margin'] * afl_avgs['elo_Opp'] / afl_avgs['elo']
afl_avgs = afl_feature_creation.create_rolling_averages(afl_avgs, 6, ['Adj_elo_ave_margin'])

# Create average elo of opponents beaten/lost to feature
afl_avgs = afl_feature_creation.create_ave_elo_opponent(afl_avgs, 6, beaten_or_lost='beaten')
afl_avgs = afl_feature_creation.create_ave_elo_opponent(afl_avgs, 6, beaten_or_lost='lost')

# Create regular margin rolling average
afl_avgs = afl_feature_creation.create_rolling_averages(afl_avgs, 6, ['Margin'])
afl_avgs.tail()

,Team,odds,Date,home_team,away_team,Behinds,Game,Goals,Home?,Opposition,Opposition Behinds,Opposition Goals,Opposition Points,Points,Round,Venue,Season,Status,GA_ave_6,CP_ave_6,UP_ave_6,ED_ave_6,CM_ave_6,MI5_ave_6,One.Percenters_ave_6,BO_ave_6,K_ave_6,HB_ave_6,D_ave_6,M_ave_6,G_ave_6,B_ave_6,T_ave_6,HO_ave_6,I50_ave_6,CL_ave_6,CG_ave_6,R50_ave_6,FF_ave_6,FA_ave_6,AF_ave_6,SC_ave_6,CCL_ave_6,SCL_ave_6,SI_ave_6,MG_ave_6,TO_ave_6,ITC_ave_6,T5_ave_6,disposal_efficiency_ave_6,form_over_opposition_6,home_elo,away_elo,elo,elo_Opp,Adj_elo_ave_margin_ave_6,average_elo_opponents_beaten_6,average_elo_opponents_lost_6,Margin_ave_6
3009,Carlton,9.80,2018-08-05,Carlton,GWS,NaN,15369,NaN,1,GWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,131.333333,224.833333,260.500000,11.500000,9.333333,43.166667,6.833333,196.166667,162.166667,358.333333,85.666667,8.666667,6.333333,59.333333,34.500000,41.500000,33.666667,55.666667,38.333333,20.000000,22.333333,1453.000000,1530.666667,12.000000,21.666667,76.166667,5001.500000,65.333333,62.333333,7.666667,0.728181,1,1254.391944,1613.612204,1254.391944,1613.612204,-40.274325,1497.756493,1459.288681,-34.500000
3010,Gold Coast,26.00,2018-08-05,Melbourne,Gold Coast,NaN,15370,NaN,0,Melbourne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.833333,146.166667,183.500000,216.333333,10.000000,8.500000,55.833333,5.333333,197.000000,133.166667,330.166667,74.833333,7.833333,12.000000,72.333333,46.833333,48.833333,36.666667,59.333333,40.333333,20.000000,21.666667,1432.000000,1493.333333,10.833333,25.833333,71.666667,5563.500000,71.000000,72.333333,12.666667,0.654760,2,1509.390579,1294.817584,1294.817584,1509.390579,-27.575515,1399.148978,1468.770338,-24.833333
3011,Melbourne,1.04,2018-08-05,Melbourne,Gold Coast,NaN,15370,NaN,1,Gold Coast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.666667,168.500000,232.000000,279.166667,11.166667,12.000000,50.333333,2.833333,216.166667,183.666667,399.833333,84.500000,15.000000,10.666667,74.833333,54.333333,62.333333,43.166667,59.833333,30.500000,19.166667,20.833333,1680.333333,1714.500000,14.833333,28.333333,117.666667,6266.833333,75.166667,78.833333,14.500000,0.694229,4,1509.390579,1294.817584,1509.390579,1294.817584,34.258681,1490.159807,1633.148523,32.666667
3012,Fremantle,7.00,2018-08-05,West Coast,Fremantle,NaN,15371,NaN,0,West Coast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,135.000000,222.166667,251.500000,6.833333,8.833333,48.000000,3.166667,199.333333,155.500000,354.833333,82.500000,9.333333,6.333333,68.500000,37.833333,44.500000,32.166667,55.000000,39.666667,19.666667,19.833333,1490.833333,1580.333333,10.000000,22.166667,80.833333,5296.000000,74.500000,70.666667,10.500000,0.708507,1,1591.711462,1426.405257,1426.405257,1591.711462,-19.687707,1442.470402,1543.556852,-17.333333
3013,West Coast,1.17,2018-08-05,West Coast,Fremantle,NaN,15371,NaN,1,Fremantle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.166667,135.833333,210.333333,255.333333,12.833333,10.166667,48.666667,3.166667,222.166667,125.166667,347.333333,98.333333,10.000000,9.000000,67.000000,41.666667,51.500000,33.500000,50.500000,38.500000,24.166667,20.333333,1553.666667,1609.833333,9.833333,23.666667,79.166667,5513.000000,71.500000,72.833333,9.333333,0.733040,5,1591.711462,1426.405257,1591.711462,1426.405257,-0.850816,1529.169608,1594.198154,-0.666667


Our DataFrame looks great! Although there are some NaNs, these are only in columns like 'Goals' which obviously don't have a value yet as the game hasn't been played.

Now we need to get our DataFrame on one line so that each row corresponds to one footy game. Let's use our previously defined function for this. Let's also drop the columns with NaNs which we don't need.

In [31]:
# Get each footy match on individual rows and drop irrelevant columns
one_line = afl_modelling.get_df_on_one_line(afl_avgs)

# Drop duplicate columns and unnecessary columns
cols_to_drop = ['Opposition Behinds', 'Goals', 'Behinds', 'Opposition Goals', 'Opposition Points', 'Points', 'Round', 'Venue', 'Season', 'Status',
               'CCL_ave_6', 'SCL_ave_6', 'SI_ave_6', 'MG_ave_6', 'TO_ave_6', 'ITC_ave_6', 'T5_ave_6', 'elo', 'elo_Opp', 'Behinds_away',
               'Goals_away', 'home_elo_away', 'away_elo_away', 'elo_away', 'elo_Opp_away']
one_line = one_line.drop(columns=cols_to_drop)
# Drop all columns where home_elo or away_elo == 1500 exactly, as this is the first game played
one_line = one_line[(one_line['home_elo'] != 1500) & (one_line['away_elo'] != 1500)]

# Drop Na rows from calculating moving averages
one_line = one_line.dropna(axis=0)

Now let's create our differential_df and then filter out DataFrame to only include this weekend's game, based on the Game IDs we created earlier

In [32]:
diff_df = afl_modelling.get_diff_df(one_line)
diff_df = diff_df.drop(columns=['odds', 'odds_away']).select_dtypes(include=[np.number])
prediction_feature_set = diff_df[diff_df['Game'].isin(game_ids_next_round)].dropna(axis=1)

In [33]:
prediction_feature_set

,Game,home_elo,away_elo,CCL_ave_6_away,SCL_ave_6_away,SI_ave_6_away,MG_ave_6_away,TO_ave_6_away,ITC_ave_6_away,T5_ave_6_away,GA_ave_6_diff,CP_ave_6_diff,UP_ave_6_diff,ED_ave_6_diff,CM_ave_6_diff,MI5_ave_6_diff,One.Percenters_ave_6_diff,BO_ave_6_diff,K_ave_6_diff,HB_ave_6_diff,D_ave_6_diff,M_ave_6_diff,G_ave_6_diff,B_ave_6_diff,T_ave_6_diff,HO_ave_6_diff,I50_ave_6_diff,CL_ave_6_diff,CG_ave_6_diff,R50_ave_6_diff,FF_ave_6_diff,FA_ave_6_diff,AF_ave_6_diff,SC_ave_6_diff,disposal_efficiency_ave_6_diff,form_over_opposition_6_diff,Adj_elo_ave_margin_ave_6_diff,average_elo_opponents_beaten_6_diff,average_elo_opponents_lost_6_diff,Margin_ave_6_diff,implied_odds_prob,implied_odds_prob_away
1526,15363,1624.637563,1643.524765,14.000000,24.833333,98.166667,5696.500000,72.500000,67.500000,11.666667,-1.166667,0.500000,-12.333333,-9.500000,2.666667,3.166667,1.833333,4.000000,3.666667,-13.166667,-9.500000,3.666667,1.166667,1.500000,-7.833333,-3.333333,5.500000,-6.500000,1.833333,-3.000000,-4.666667,0.500000,-36.666667,34.500000,-0.004187,-2,22.533546,99.922029,42.354317,22.333333,0.684932,0.322581
1527,15364,1590.377405,1528.418023,12.000000,29.166667,107.333333,6049.166667,68.833333,75.166667,14.833333,0.166667,-14.000000,28.666667,26.833333,-2.500000,1.166667,3.333333,-4.000000,-14.333333,25.166667,10.833333,-0.500000,1.666667,-1.666667,-1.833333,-3.666667,-3.833333,-8.666667,-2.333333,-7.333333,-0.333333,-1.166667,6.333333,-51.666667,0.048052,2,-7.727833,-121.912760,-38.154835,-1.666667,0.467290,0.531915
1528,15365,1280.219510,1472.910710,14.166667,23.666667,89.500000,5442.166667,71.333333,73.666667,13.666667,0.833333,-14.833333,23.666667,26.166667,-2.166667,2.166667,8.833333,0.833333,23.333333,-9.500000,13.833333,22.500000,-0.333333,0.333333,-6.833333,1.500000,0.166667,1.833333,0.166667,5.666667,2.500000,2.166667,87.000000,55.833333,0.041852,-4,-9.697430,-136.424997,59.200353,-5.833333,0.540541,0.458716
1529,15366,1620.104725,1563.944697,10.000000,27.166667,81.000000,5807.333333,75.666667,78.166667,16.666667,-2.500000,3.500000,-22.000000,-10.500000,-1.666667,-3.833333,-13.333333,-4.666667,0.166667,-14.166667,-14.000000,-7.833333,-1.000000,0.000000,-6.000000,-2.666667,-4.000000,1.166667,-5.500000,-4.166667,-3.833333,-2.500000,-80.333333,-127.500000,-0.003273,4,-28.195956,43.107540,-58.274674,-31.333333,0.458716,0.537634
1530,15367,1416.739182,1436.609139,9.833333,26.000000,66.333333,5279.833333,77.166667,71.333333,14.166667,1.000000,-2.333333,21.833333,12.000000,0.833333,2.666667,-7.166667,1.833333,14.500000,1.833333,16.333333,19.666667,0.833333,3.666667,-3.000000,4.833333,2.000000,1.833333,-0.833333,-1.000000,1.666667,-1.833333,114.833333,90.666667,-0.000853,-2,20.657500,-61.384292,35.047135,22.333333,0.595238,0.403226
1531,15368,1630.736013,1501.449238,11.500000,25.333333,97.500000,5769.833333,74.833333,76.833333,8.500000,-2.666667,7.500000,-41.333333,-37.000000,-0.333333,-2.666667,-7.666667,-2.333333,-13.833333,-17.833333,-31.666667,-15.500000,-2.333333,-1.333333,1.833333,-13.166667,-6.000000,-1.166667,1.166667,11.000000,-2.333333,5.166667,-162.666667,-109.333333,-0.035860,0,-22.115997,93.684897,-108.788841,-25.166667,0.568182,0.427350
1532,15369,1254.391944,1613.612204,12.166667,31.000000,109.000000,6164.166667,71.166667,72.500000,11.333333,-2.333333,-23.666667,-0.666667,-12.666667,-0.500000,-0.833333,-11.833333,-1.500000,-24.333333,4.000000,-20.333333,-2.166667,-3.333333,-3.833333,-1.666667,-6.833333,-13.166667,-9.500000,1.666667,-5.000000,-1.333333,1.666667,-115.166667,-163.166667,0.006805,-3,-47.990726,45.357704,-110.018063,-43.166667,0.102041,0.892857
1533,15370,1509.390579,1294.817584,10.833333,25.833333,71.666667,5563.500000,71.000000,72.333333,12.666667,6.833333,22.333333,48.500000,62.833333,1.166667,3.500000,-5.500000,-2.500000,19.166667,50.500000,69.666667,9.666667,7.166667,-1.333333,2.500000,7.500000,13.500000,6.500000,0.500000,-9.833333,-0.833333,-0.833333,248.333333,221.166667,0.039469,2,61.834195,91.010828,164.378185,57.5000

Great! We now have our feature DataFrame for this weekend's games. Let's now model this to create our predictions

## Creating Our Predictions
To create our predictions we will use similar code to what we used in our AFL Modelling Tutorial. We will need to train our model on all the data we have (up until last week's games), and then use our trained model to predict this week. Again, we will use Stacking with XGB, like we did in our AFL Modelling Tutorial.

In [34]:
afl = prepare_afl_features(window=6, k_factor=24).dropna().sort_values(by='Game')
afl = afl_modelling.get_df_on_one_line(afl)

# Drop columns which leak data and which we don't need
dropped_cols = ['Behinds', 'Goals', 'Opposition Behinds', 'Opposition Goals', 'Opposition Points', 'Points',
               'elo', 'elo_Opp', 'CCL_ave_6', 'SCL_ave_6', 'SI_ave_6', 'MG_ave_6', 'TO_ave_6', 'ITC_ave_6', 'T5_ave_6', 'home_win_away',
               'home_elo_away', 'away_elo_away']
afl = afl.drop(columns=dropped_cols)
# Get a differential DataFrame - subtracting the away features from the home features
diff_df = afl_modelling.get_diff_df(afl)

In [35]:
# Hard code estimators from our last tutorial
all_estimators = [
    LogisticRegression(C=0.75, class_weight=None, dual=False, fit_intercept=True,
       intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
       penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
       verbose=0, warm_start=False),
    
    RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
     max_iter=None, normalize=False, random_state=None, solver='auto',
     tol=0.001),
    
    RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
         max_depth=15, max_features='sqrt', max_leaf_nodes=None,
         min_impurity_decrease=0.0, min_impurity_split=None,
         min_samples_leaf=1, min_samples_split=10,
         min_weight_fraction_leaf=0.0, n_estimators=750, n_jobs=-1,
         oob_score=False, random_state=5, verbose=0, warm_start=False),
    
    GaussianNB(priors=None),
    
    LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
           solver='svd', store_covariance=False, tol=0.0001)
]

# Hard code best cols
best_cols = ['home_elo',
'away_elo',
'GA_ave_6_diff',
'CP_ave_6_diff',
'UP_ave_6_diff',
'ED_ave_6_diff',
'CM_ave_6_diff',
'MI5_ave_6_diff',
'One.Percenters_ave_6_diff',
'BO_ave_6_diff',
'HB_ave_6_diff',
'M_ave_6_diff',
'G_ave_6_diff',
'T_ave_6_diff',
'HO_ave_6_diff',
'I50_ave_6_diff',
'CL_ave_6_diff',
'CG_ave_6_diff',
'R50_ave_6_diff',
'FF_ave_6_diff',
'FA_ave_6_diff',
'AF_ave_6_diff',
'SC_ave_6_diff',
'disposal_efficiency_ave_6_diff',
'R50_efficiency_ave_6_diff',
'I50_efficiency_ave_6_diff',
'Adj_elo_ave_margin_ave_6_diff',
'average_elo_opponents_beaten_6_diff',
'average_elo_opponents_lost_6_diff',
'Margin_ave_6_diff',
'implied_odds_prob',
'implied_odds_prob_away']

In [36]:
# Create our train sets
X = diff_df.drop(columns=['home_win']).select_dtypes(include=[np.number])
y = diff_df['home_win']

features = prediction_feature_set.columns

# Predict Next Round
preds_next_round = afl_modelling.implement_xgb_stacking(X[features].drop(columns='Game'), y, 
                                                        prediction_feature_set.drop(columns='Game'), all_estimators)

C:\Users\wardj\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Now that we have our predictions, the final step is to put our predictions into a DataFrame so they're easy on the eye, and our predictions correspond to actual home and away teams rather than Game IDs. This is the final step.

In [37]:
preds_df = pd.DataFrame({
    "Game": prediction_feature_set['Game'],
    "Prediction (home_win)": preds_next_round
})

prediction_feature_set['home_odds'] = 1 / prediction_feature_set['implied_odds_prob']
prediction_feature_set['away_odds'] = 1 / prediction_feature_set['implied_odds_prob_away']

final_preds_df = pd.merge(preds_df, next_week_df[['home_team', 'away_team', 'Game']], on='Game').drop_duplicates()
final_preds_df = pd.merge(final_preds_df, prediction_feature_set[['home_odds', 'away_odds', 'home_elo', 'away_elo', 'Game']], on='Game')
final_preds_df['Predicted Winner'] = final_preds_df.apply(lambda x: x['home_team'] if x['Prediction (home_win)'] == 1 else x['away_team'], axis=1)

In [42]:
final_preds_df[['home_team', 'away_team', 'Predicted Winner', 'home_odds', 'away_odds', 'home_elo', 'away_elo']]

,home_team,away_team,Predicted Winner,home_odds,away_odds,home_elo,away_elo
0,Richmond,Geelong,Richmond,1.46,3.10,1624.637563,1643.524765
1,Hawthorn,Essendon,Hawthorn,2.14,1.88,1590.377405,1528.418023
2,Brisbane,North Melbourne,Brisbane,1.85,2.18,1280.219510,1472.910710
3,Adelaide,Port Adelaide,Adelaide,2.18,1.86,1620.104725,1563.944697
4,St Kilda,Western Bulldogs,St Kilda,1.68,2.48,1416.739182,1436.609139
5,Sydney,Collingwood,Sydney,1.76,2.34,1630.736013,1501.449238
6,Carlton,GWS,GWS,9.80,1.12,1254.391944,1613.612204
7,Melbourne,Gold Coast,Melbourne,1.04,26.00,1509.390579,1294.817584
8,West Coast,Fremantle,West Coast,1.17,7.00,1591.711462,1426.405257


## Conclusion
Congratulations! You have created AFL predictions for this week. If you are beginner to this, don't be overwhelmed. The process gets easier each time you do it. And it is super rewarding. In future iterations we will update this tutorial to predict actual odds, and then integrate this with Betfair's API so that you can create an automated betting strategy using Machine Learning to create your predictions!